In [70]:
pip install plotly

StatementMeta(, d3e5e4e7-b83c-4efd-882d-2180a1aa621f, 72, Finished, Available, Finished)

Note: you may need to restart the kernel to use updated packages.


In [71]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from pyspark.sql import SparkSession

StatementMeta(, d3e5e4e7-b83c-4efd-882d-2180a1aa621f, 73, Finished, Available, Finished)

In [72]:
categoria = spark.sql("SELECT * FROM LakeHouse_Murilo.categoria LIMIT 1000")
display(categoria)

StatementMeta(, d3e5e4e7-b83c-4efd-882d-2180a1aa621f, 74, Finished, Available, Finished)

DataFrame[IDCATEGORIA: int, NOME: string]

In [73]:
produto = spark.sql("SELECT * FROM LakeHouse_Murilo.produto LIMIT 1000")
display(produto)

StatementMeta(, d3e5e4e7-b83c-4efd-882d-2180a1aa621f, 75, Finished, Available, Finished)

DataFrame[IDSK: int, IDPRODUTO: int, NOME: string, ID_CATEGORIA: int, CUSTO_MEDIO: double, VALOR_UNITARIO: double]

In [74]:
df_categoria_pandas = categoria.toPandas()
df_produto_pandas = produto.toPandas()

StatementMeta(, d3e5e4e7-b83c-4efd-882d-2180a1aa621f, 76, Finished, Available, Finished)

In [75]:
df_categoria_pandas.head()

StatementMeta(, d3e5e4e7-b83c-4efd-882d-2180a1aa621f, 77, Finished, Available, Finished)

,IDCATEGORIA,NOME
0,1,LIVROS
1,2,DVDS
2,3,GAMES
3,4,CELULARES
4,5,INFORMATICA


In [76]:
df_produto_pandas.head()

StatementMeta(, d3e5e4e7-b83c-4efd-882d-2180a1aa621f, 78, Finished, Available, Finished)

,IDSK,IDPRODUTO,NOME,ID_CATEGORIA,CUSTO_MEDIO,VALOR_UNITARIO
0,1,1,Cavaleiro Dos 7 Reinos,1,15.0,30.0
1,2,2,Jogos Vorazes,1,22.0,45.0
2,3,3,Revelados,1,24.0,48.0
3,4,4,A Culpa É Das Estrelas,1,12.0,24.0
4,5,5,Um Caso Perdido,1,20.0,56.0


In [77]:
%%sql
SELECT
    p.NOME AS produto,
    p.CUSTO_MEDIO AS custo_medio,
    p.VALOR_UNITARIO AS valor_unitario,
    c.NOME AS categoria
FROM 
    produto p
INNER JOIN 
    categoria c ON p.ID_CATEGORIA = c.IDCATEGORIA;

StatementMeta(, d3e5e4e7-b83c-4efd-882d-2180a1aa621f, 79, Finished, Available, Finished)

<Spark SQL result set with 234 rows and 4 fields>

In [78]:
# Realize o merge (join) com base nas colunas relacionais
df_join = pd.merge(
    left=df_produto_pandas,
    right=df_categoria_pandas,
    how='inner',
    left_on='ID_CATEGORIA',  # Coluna do produto
    right_on='IDCATEGORIA'  # Coluna da categoria
)

# Renomeie as colunas para melhor legibilidade (opcional)
df_join.rename(columns={
    'NOME_x': 'Produto',
    'NOME_y': 'Categoria',
    'CUSTO_MEDIO': 'Custo Médio',
    'VALOR_UNITARIO': 'Valor Unitário'
}, inplace=True)

# Exiba os primeiros 300 registros
display(df_join[['Produto', 'Categoria', 'Custo Médio', 'Valor Unitário']].head(300))

StatementMeta(, d3e5e4e7-b83c-4efd-882d-2180a1aa621f, 80, Finished, Available, Finished)

,Produto,Categoria,Custo Médio,Valor Unitário
0,Cavaleiro Dos 7 Reinos,LIVROS,15.0,30.0
1,Jogos Vorazes,LIVROS,22.0,45.0
2,Revelados,LIVROS,24.0,48.0
3,A Culpa É Das Estrelas,LIVROS,12.0,24.0
4,Um Caso Perdido,LIVROS,20.0,56.0
...,...,...,...,...
229,Rio De Janeiro / Londrina,PASSAGENS,390.0,430.0
230,Rio De Janeiro / Madrid,PASSAGENS,1623.0,1800.0
231,Rio De Janeiro / Paris,PASSAGENS,1980.0,2300.0
232,Rio De Janeiro / Miami,PASSAGENS,1763.0,1900.0


In [79]:
# Realize o merge (join) com base nas colunas relacionais
df_join2 = pd.merge(
    left=df_produto_pandas,
    right=df_categoria_pandas,
    how='inner',
    left_on='ID_CATEGORIA',  # Coluna do produto
    right_on='IDCATEGORIA',  # Coluna da categoria
    suffixes=('_prod', '_cat')
)

# Exiba os primeiros 300 registros
df_join2.head(300)

StatementMeta(, d3e5e4e7-b83c-4efd-882d-2180a1aa621f, 81, Finished, Available, Finished)

,IDSK,IDPRODUTO,NOME_prod,ID_CATEGORIA,CUSTO_MEDIO,VALOR_UNITARIO,IDCATEGORIA,NOME_cat
0,1,1,Cavaleiro Dos 7 Reinos,1,15.0,30.0,1,LIVROS
1,2,2,Jogos Vorazes,1,22.0,45.0,1,LIVROS
2,3,3,Revelados,1,24.0,48.0,1,LIVROS
3,4,4,A Culpa É Das Estrelas,1,12.0,24.0,1,LIVROS
4,5,5,Um Caso Perdido,1,20.0,56.0,1,LIVROS
...,...,...,...,...,...,...,...,...
229,228,228,Rio De Janeiro / Londrina,9,390.0,430.0,9,PASSAGENS
230,229,229,Rio De Janeiro / Madrid,9,1623.0,1800.0,9,PASSAGENS
231,230,230,Rio De Janeiro / Paris,9,1980.0,2300.0,9,PASSAGENS
232,231,231,Rio De Janeiro / Miami,9,1763.0,1900.0,9,PASSAGENS


In [80]:
join = {
    'Produto': df_produto_pandas['NOME'],
    'Categoria': df_categoria_pandas['NOME'],
    'Custo Médio': df_produto_pandas['CUSTO_MEDIO'],
    'Valor Unitário': df_produto_pandas['VALOR_UNITARIO']
}
join_view = pd.DataFrame(join)
join_view.head(300)

StatementMeta(, d3e5e4e7-b83c-4efd-882d-2180a1aa621f, 82, Finished, Available, Finished)

,Produto,Categoria,Custo Médio,Valor Unitário
0,Cavaleiro Dos 7 Reinos,LIVROS,15.0,30.0
1,Jogos Vorazes,DVDS,22.0,45.0
2,Revelados,GAMES,24.0,48.0
3,A Culpa É Das Estrelas,CELULARES,12.0,24.0
4,Um Caso Perdido,INFORMATICA,20.0,56.0
...,...,...,...,...
229,Rio De Janeiro / Paris,NaN,1980.0,2300.0
230,Rio De Janeiro / Miami,NaN,1763.0,1900.0
231,Rio De Janeiro / New York,NaN,2354.0,2600.0
232,Moto Z Play 3,NaN,900.0,1150.0


In [81]:
# Criar gráfico de dispersão
fig = px.scatter(
    df_join,
    x='Categoria',             # Eixo X
    y='Valor Unitário',          # Eixo Y
    color='Categoria',           # Cor por categoria
    size='Valor Unitário',          # Tamanho dos pontos conforme custo médio
    title='Preços dos Produtos por Categoria',
    labels={
        'Categoria': 'Categorias',
        'Valor Unitário': 'Preço(R$)'
    },
    hover_name='Produto'         # Mostra o nome do produto ao passar o mouse
)

# Exibir o gráfico interativo
fig.show()

StatementMeta(, d3e5e4e7-b83c-4efd-882d-2180a1aa621f, 83, Finished, Available, Finished)

In [89]:
df_join.to_csv('Categorias.csv', sep=';')

StatementMeta(, d3e5e4e7-b83c-4efd-882d-2180a1aa621f, 91, Finished, Available, Finished)

In [90]:
#1 Criar DataFrame Spark
spark = SparkSession.builder.getOrCreate()
df_spark = spark.createDataFrame(df_join)

caminho_da_tabela = "Tables/Dispersao.csv"
df_spark.write.mode("overwrite").option("header", "true").csv(caminho_da_tabela)


StatementMeta(, d3e5e4e7-b83c-4efd-882d-2180a1aa621f, 92, Finished, Available, Finished)

Py4JJavaError: An error occurred while calling o20206.csv.
: java.nio.file.AccessDeniedException: Operation failed: "Forbidden", 403, DELETE, http://onelake.dfs.fabric.microsoft.com/4d610067-8093-41e5-8082-7cad66a1a874/880d56b1-edb6-446a-b2c6-52a7e7822dfe/Files?timeout=90&recursive=true, AuthorizationFailure, "This request is not authorized to perform this operation."
	at org.apache.hadoop.fs.azurebfs.AzureBlobFileSystem.checkException(AzureBlobFileSystem.java:1443)
	at org.apache.hadoop.fs.azurebfs.AzureBlobFileSystem.delete(AzureBlobFileSystem.java:496)
	at org.apache.spark.internal.io.FileCommitProtocol.deleteWithJob(FileCommitProtocol.scala:184)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.deleteMatchingPartitions(InsertIntoHadoopFsRelationCommand.scala:239)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:131)
	at org.apache.spark.sql.execution.command.DataWritingCommand